### Data Scraping and Cleaning
This code will scrape Top 90 ODI batsman's data from espncricinfo stats page. We will be needing both innings by innings list and cumulative data of the batsmans. First we need player id's which then will be used to get individual player data. We will be using BeautifulSoup to scrape the data

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import csv

In [2]:
# get the page using requests
players_page = requests.get("https://stats.espncricinfo.com/ci/content/records/83548.html")

# parse the data
soup = BeautifulSoup(players_page.text, 'html.parser')
players_list = soup.find_all("a", attrs={'class':'data-link', 'href': True})

# save it into a csv file
with open('players.csv', "w") as csv_file:
    for row in players_list:
        csv_file.write(row['href'][19:-5])
        csv_file.write('\n')
players_list = np.loadtxt('players.csv', delimiter=',').astype(int)
players_list

array([ 35320,  50710,   7133,  49209,  49289, 253802,  40570,  45789,
        28779,  28114,  28081,  51880,  52337,  48472,  43650,   5390,
        44936,  26329,  48462,  34102,  42605,  51469,  36084,  52047,
        38699,  48124,   8189,  35263,  43906,  45224,  42639,  37000,
         4578,  24598,  56194,   8192,  42657,  50244,  40879,  43652,
        42623,  36185,  47270,  50747, 226492,   4144,  45813,  55427,
        52812,  55814,  41434,  56143,  56029,  55429,  40551,   4174,
        52810, 277906,   5616, 303669,  28235,  37737,   6044,   4169,
        46973,  42420,  49764,  52969,  52066,   8180,  55608,  33335,
        52983,  44485,  44828, 219889,   5939,   9062,  29632, 379143,
         6513,  28763,   5334,  55301,  49626,  51901,  41378,  44932,
        10772,   7702,  39836])

Now as we have the id's we can scrape their individual innings by innings data

We use `scrapper.py` file which has some utility functions to scrap players individual data.

In [3]:
from scrapper import Player

But there is a slight problem when using this. Not all players have data on no of 4's and 6's so we run into issues if append them all together. We need remove the 4's and 6's from the players data for those available

In [4]:
# we instantiate Player objects 
player_objects = []
for player in players_list:
    player_objects.append(Player(player))

In [6]:
# we add both match by match list and cumulative scores
for player in player_objects:
    player.get_data(match_format=2, data_type='batting', view='cumulative')
    player.get_data(match_format=2, data_type='batting', view='match')

In [7]:
# we should seperate the players with 4's data and others
with_4s = []
without_4s = []
for player in player_objects:
    if(player.data["cumulative_2_batting"][0][11] == '4s'):
        with_4s.append(player)
    else:
        without_4s.append(player)

In [8]:
len(with_4s) + len(without_4s) == len(player_objects)

True

This seperation is important as it helps in easier cleaning of the data. We now should save these player data seperately and then clean them individually and remove non intersecting rows. Then we can merge them and make our dataset.

In [10]:
with open('With_4s_data.csv', "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    i= 0
    for player in with_4s:
        j = 0
        for r in range(len(player.data["cumulative_2_batting"])):
            j+=1
            if(j==1 and i==0):
                writer.writerow(player.data["cumulative_2_batting"][r] + ['player_id'] + player.data["match_2_batting"][r] + ['player_id'])
            elif (j==1 and i!=0):
                pass
            else:
                writer.writerow(player.data["cumulative_2_batting"][r] + [player.player_id] + player.data["match_2_batting"][r] + [player.player_id])
        i+=1
        
with open('Without_4s_data.csv', "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    i= 0
    for player in without_4s:
        j = 0
        for r in range(len(player.data["cumulative_2_batting"])):
            j+=1
            if(j==1 and i==0):
                writer.writerow(player.data["cumulative_2_batting"][r] + ['player_id'] + player.data["match_2_batting"][r] + ['player_id'])
            elif (j==1 and i!=0):
                pass
            else:
                writer.writerow(player.data["cumulative_2_batting"][r] + [player.player_id] + player.data["match_2_batting"][r] + [player.player_id])
        i+=1

In the above cell we save 2 csv files with each containg data of both cumulative and match by match record. We are doing some manipulating so we get uniform rows and columns. We can now clean the data using pandas such that finally both the df's match. 

In [11]:
import pandas as pd

Let's start with the data which contains the data for boundaries

In [19]:
df_4 = pd.read_csv('With_4s_data.csv')
df_4.head()

,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,...,BF.1,SR.1,4s.1,6s.1,Unnamed: 24,Opposition.1,Ground,Start Date.1,match_id.1,player_id.1
0,1,1,0,0,0,0.00,2,0.00,0,0,...,2,0.00,0,0,NaN,v Pakistan,Gujranwala,18 Dec 1989,ODI # 593,35320
1,2,2,0,0,0,0.00,4,0.00,0,0,...,2,0.00,0,0,NaN,v New Zealand,Dunedin,1 Mar 1990,ODI # 612,35320
2,3,3,0,36,36,12.00,43,83.72,0,0,...,39,92.30,5,0,NaN,v New Zealand,Wellington,6 Mar 1990,ODI # 616,35320
3,4,4,0,46,36,11.50,55,83.63,0,0,...,12,83.33,0,0,NaN,v Sri Lanka,Sharjah,25 Apr 1990,ODI # 623,35320
4,5,5,0,66,36,13.20,80,82.50,0,0,...,25,80.00,1,0,NaN,v Pakistan,Sharjah,27 Apr 1990,ODI # 625,35320


In [20]:
col = df_4.columns.values.tolist()
col

['Mat',
 'Inns',
 'NO',
 'Runs',
 'HS',
 'Ave',
 'BF',
 'SR',
 '100',
 '50',
 '0',
 '4s',
 '6s',
 'Unnamed: 13',
 'Opposition',
 'Start Date',
 'match_id',
 'player_id',
 'Bat1',
 'Runs.1',
 'BF.1',
 'SR.1',
 '4s.1',
 '6s.1',
 'Unnamed: 24',
 'Opposition.1',
 'Ground',
 'Start Date.1',
 'match_id.1',
 'player_id.1']

In [21]:
df_4.drop(['Unnamed: 13', '4s', '6s', 'Start Date', '4s.1', '6s.1', 'Start Date.1', 'Opposition.1', 'match_id.1', 'player_id.1', 'Unnamed: 24', 'Inns', 'Runs.1', 'BF.1'], axis=1, inplace=True)
df_4.head()

,Mat,NO,Runs,HS,Ave,BF,SR,100,50,0,Opposition,match_id,player_id,Bat1,SR.1,Ground
0,1,0,0,0,0.00,2,0.00,0,0,1,v Pakistan,ODI # 593,35320,0,0.00,Gujranwala
1,2,0,0,0,0.00,4,0.00,0,0,2,v New Zealand,ODI # 612,35320,0,0.00,Dunedin
2,3,0,36,36,12.00,43,83.72,0,0,2,v New Zealand,ODI # 616,35320,36,92.30,Wellington
3,4,0,46,36,11.50,55,83.63,0,0,2,v Sri Lanka,ODI # 623,35320,10,83.33,Sharjah
4,5,0,66,36,13.20,80,82.50,0,0,2,v Pakistan,ODI # 625,35320,20,80.00,Sharjah


In [22]:
df_4.rename(columns={'Runs': 'Total Runs', 'BF': 'Total Balls', 'SR': 'Career SR', 'Bat1': 'Match Runs', 'SR.1': 'Match SR'}, inplace=True)
df_4.head()

,Mat,NO,Total Runs,HS,Ave,Total Balls,Career SR,100,50,0,Opposition,match_id,player_id,Match Runs,Match SR,Ground
0,1,0,0,0,0.00,2,0.00,0,0,1,v Pakistan,ODI # 593,35320,0,0.00,Gujranwala
1,2,0,0,0,0.00,4,0.00,0,0,2,v New Zealand,ODI # 612,35320,0,0.00,Dunedin
2,3,0,36,36,12.00,43,83.72,0,0,2,v New Zealand,ODI # 616,35320,36,92.30,Wellington
3,4,0,46,36,11.50,55,83.63,0,0,2,v Sri Lanka,ODI # 623,35320,10,83.33,Sharjah
4,5,0,66,36,13.20,80,82.50,0,0,2,v Pakistan,ODI # 625,35320,20,80.00,Sharjah


In [23]:
df = pd.read_csv('Without_4s_data.csv')
df.head()

,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,...,Bat1,Runs.1,BF.1,SR.1,Unnamed: 20,Opposition.1,Ground,Start Date.1,match_id.1,player_id.1
0,1,1,1,47,47*,-,37,127.02,0,0,...,47*,47,37,127.02,NaN,v England,Bengaluru,20 Jan 1985,ODI # 293,26329
1,2,2,1,94,47*,94.00,92,102.17,0,0,...,47,47,55,85.45,NaN,v England,Nagpur,23 Jan 1985,ODI # 295,26329
2,3,3,1,104,47*,52.00,103,100.97,0,0,...,10,10,11,90.90,NaN,v England,Chandigarh,27 Jan 1985,ODI # 298,26329
3,4,4,2,197,93*,98.50,238,82.77,0,1,...,93*,93,135,68.88,NaN,v Pakistan,Melbourne,20 Feb 1985,ODI # 309,26329
4,5,5,2,242,93*,80.66,305,79.34,0,1,...,45,45,67,67.16,NaN,v England,Sydney,26 Feb 1985,ODI # 312,26329


In [26]:
df.drop(['Unnamed: 11', 'Start Date', 'Start Date.1', 'Opposition.1', 'match_id.1', 'player_id.1', 'Unnamed: 20', 'Inns', 'Runs.1', 'BF.1'], axis=1, inplace=True)
df.rename(columns={'Runs': 'Total Runs', 'BF': 'Total Balls', 'SR': 'Career SR', 'Bat1': 'Match Runs', 'SR.1': 'Match SR'}, inplace=True)
df.head()

,Mat,NO,Total Runs,HS,Ave,Total Balls,Career SR,100,50,0,Opposition,match_id,player_id,Match Runs,Match SR,Ground
0,1,1,47,47*,-,37,127.02,0,0,0,v England,ODI # 293,26329,47*,127.02,Bengaluru
1,2,1,94,47*,94.00,92,102.17,0,0,0,v England,ODI # 295,26329,47,85.45,Nagpur
2,3,1,104,47*,52.00,103,100.97,0,0,0,v England,ODI # 298,26329,10,90.90,Chandigarh
3,4,2,197,93*,98.50,238,82.77,0,1,0,v Pakistan,ODI # 309,26329,93*,68.88,Melbourne
4,5,2,242,93*,80.66,305,79.34,0,1,0,v England,ODI # 312,26329,45,67.16,Sydney


In [27]:
df_4.append(df)
df_4.to_csv('cleaned_data.csv')

So cleaning is done and we now have a dataset which we can use to train right? No, There are still some things to be done. Like * in Runs and HS column. DNB and TDNB in Match Runs column. removing v in the Opposition column and removing ODI # in the match_id etc

In [45]:
df = pd.read_csv('cleaned_data.csv')
df.head()

,Unnamed: 0,Mat,NO,Total Runs,HS,Ave,Total Balls,Career SR,100,50,0,Opposition,match_id,player_id,Match Runs,Match SR,Ground
0,0,1,0,0,0,0.00,2,0.00,0,0,1,v Pakistan,ODI # 593,35320,0,0.00,Gujranwala
1,1,2,0,0,0,0.00,4,0.00,0,0,2,v New Zealand,ODI # 612,35320,0,0.00,Dunedin
2,2,3,0,36,36,12.00,43,83.72,0,0,2,v New Zealand,ODI # 616,35320,36,92.30,Wellington
3,3,4,0,46,36,11.50,55,83.63,0,0,2,v Sri Lanka,ODI # 623,35320,10,83.33,Sharjah
4,4,5,0,66,36,13.20,80,82.50,0,0,2,v Pakistan,ODI # 625,35320,20,80.00,Sharjah


In [46]:
df['Opposition'] = df['Opposition'].str.slice(2)
df['match_id'] = df['match_id'].str.slice(6)
df.head()

,Unnamed: 0,Mat,NO,Total Runs,HS,Ave,Total Balls,Career SR,100,50,0,Opposition,match_id,player_id,Match Runs,Match SR,Ground
0,0,1,0,0,0,0.00,2,0.00,0,0,1,Pakistan,593,35320,0,0.00,Gujranwala
1,1,2,0,0,0,0.00,4,0.00,0,0,2,New Zealand,612,35320,0,0.00,Dunedin
2,2,3,0,36,36,12.00,43,83.72,0,0,2,New Zealand,616,35320,36,92.30,Wellington
3,3,4,0,46,36,11.50,55,83.63,0,0,2,Sri Lanka,623,35320,10,83.33,Sharjah
4,4,5,0,66,36,13.20,80,82.50,0,0,2,Pakistan,625,35320,20,80.00,Sharjah


In [47]:
s = df["Ave"].eq("-")
df.loc[s, "Ave"] = np.nan
df["Ave"].ffill(inplace=True)
df.loc[df['Ave'] == '-', 'Ave'].count()

0

In [49]:
df = df[df['Match Runs'] != 'DNB']
df = df[df['Match Runs'] != 'TDNB']
df.loc[df['Match Runs'] == 'DNB', 'Match Runs'].count()

0

In [50]:
df.drop(['Unnamed: 0', 'Mat'], axis=1, inplace=True)
df = df[['Match Runs', 'Match SR', 'Total Runs', 'Total Balls', 'Ave', 'Career SR', 'HS', 'NO', '0', '50', '100', 'Opposition', 'Ground', 'match_id', 'player_id']]

In [51]:
df.to_csv('Filled_data.csv')
df.head()

,Match Runs,Match SR,Total Runs,Total Balls,Ave,Career SR,HS,NO,0,50,100,Opposition,Ground,match_id,player_id
0,0,0.00,0,2,0.00,0.00,0,0,1,0,0,Pakistan,Gujranwala,593,35320
1,0,0.00,0,4,0.00,0.00,0,0,2,0,0,New Zealand,Dunedin,612,35320
2,36,92.30,36,43,12.00,83.72,36,0,2,0,0,New Zealand,Wellington,616,35320
3,10,83.33,46,55,11.50,83.63,36,0,2,0,0,Sri Lanka,Sharjah,623,35320
4,20,80.00,66,80,13.20,82.50,36,0,2,0,0,Pakistan,Sharjah,625,35320


We haved removed '-', DNB and TNDB from the dataset. This completes the cleaning part of the data. We now to engineer some features, drop some columns and make a 5-match form data using rolling window method. 